# 21. Map Paved Shoulders Dashcam

Draw maps to compare the bicycle lane routes detected from dashcam images in a survey area vs. the OpenStreetMap cycleways

## Configuration

Any configuration that is required to run this notebook can be customized in the next cell

In [1]:
# Name of the subdirectory containing dashcam footage for an area, split into frame images in a
# "split" subdirectory, with an associated "metadata.csv"
# This subdirectory is assumed to be in the 'data_sources' directory
import_directory = 'dashcam_tour_mount_eliza'

# Which "locality" do we wish to process?
locality = 'Mount Eliza'

# We will sample the middle of each intersection, but we can also sample a
# "margin" around the intersection, at 10m intervals.
# E.g. if we set this to "20" then we will sample points at:
#    -20m, -10m, 0m, 10m, and 20m
# from the centre of the intersection, along the assumed bearing of the road
margin = 20

# Default zoom level for maps
zoom = 12

## Code

In [2]:
# General imports
import json
import os
import sys

from ipyleaflet import Map, Marker, GeoJSON

import pandas as pd

# Make sure local modules can be imported
module_path_root = os.path.abspath(os.pardir)
if module_path_root not in sys.path:
    sys.path.append(module_path_root)
    
# Import local modules
import osm_gsv_utils.detection_map as detection_map

In [3]:
# Derive paths for configuration

# A version of the locality name with spaces replaced by underscores
locality_clean   = locality.replace(' ', '_')

# Name of the locality with the margin around intersections included
locality_margin  = '{0:s}_{1:d}m'.format(locality_clean, margin)

# Detection directory
detection_dir = os.path.join(module_path_root, 'data_sources', import_directory, 'detections')

# Geojson files of detectsions and comparisons to OSM
detected_filename = os.path.join(detection_dir, 'hit.geojson')
tag_filename      = os.path.join(detection_dir, 'tag.geojson')
both_filename     = os.path.join(detection_dir, 'both.geojson')
hit_only_filename = os.path.join(detection_dir, 'hit_only.geojson')
tag_only_filename = os.path.join(detection_dir, 'tag_only.geojson')

# Geojson file describing the outline of the survey area
margin_filename   = os.path.join(module_path_root, 'data_sources', 'Locality_{0:s}.geojson'.format(locality_clean))

# Geojson file describing the exact outline of the town
LGA_locality = locality.replace(' Sample', '')
locality_filename = os.path.join(os.pardir, 'data_sources', 'LGA_boundary_' + LGA_locality + '.geojson')

# Geojson file with detected paved shoulders
lanes_filename = os.path.join(module_path_root, 'data_sources', 'dashcam_tour_mount_eliza', 'split', 'lanes.geojson')

## Map 1: Detected bicycle lane routes

Infer bicycle lane routes based on the points at which bicycle lane markings were detected by the model

In [4]:
# Load boundary of the survey area, so we can draw it on the map
margin_layer, margin_data = detection_map.load_layer(margin_filename)

# Find the center of the survey area, so we can center the map
center = detection_map.get_centroid(margin_data)

# Load lines that we drew from the detected points
shoulder_layer, shoulder_data = detection_map.load_layer(lanes_filename, color='red')

# Build the map from the boundary of the survey area, and the detected lanes layer
m1 = Map(center=center, zoom=zoom)
m1.add_layer(margin_layer)
m1.add_layer(shoulder_layer)

# Display the map
m1

Map(center=[-38.193788999999995, 145.094876], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom…